# Simple agent

- search tool
- bind search tool to LLM
- Use output of tools to get final answer of model
- streaming of tokens
- memory using a thread_id

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


os.environ.get("TAVILY_API_KEY=")

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [8]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
#model = ChatAnthropic(model_name="claude-3-sonnet-20240229")

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

search = TavilySearchResults(max_results=2,)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [9]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Paco! and i live in Barcelona")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im Paco! and i live in Barcelona
================================== Ai Message ==================================

Hello Paco! It's great to meet you. How can I assist you today?


In [10]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_ZeuuZJPCjNzF7lc6ZxdcwDCO)
 Call ID: call_ZeuuZJPCjNzF7lc6ZxdcwDCO
  Args:
    query: current weather in Barcelona
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in Barcelona", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Barcelona', 'region': 'Catalonia', 'country': 'Spain', 'lat': 41.3833, 'lon': 2.1833, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1743095556, 'localtime': '2025-03-27 18:12'}, 'current': {'last_updated_epoch': 1743094800, 'last_updated': '2025-03-27 18:00', 'temp_c': 14.2, 'temp_f': 57.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

# a search engine. 

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Mexico City")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in Mexico City', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Mexico City', 'region': 'The Federal District', 'country': 'Mexico', 'lat': 19.4285, 'lon': -99.1277, 'tz_id': 'America/Mexico_City', 'localtime_epoch': 1741788931, 'localtime': '2025-03-12 08:15'}, 'current': {'last_updated_epoch': 1741788000, 'last_updated': '2025-03-12 08:00', 'temp_c': 14.6, 'temp_f': 58.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 201, 'wind_dir': 'SSW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 63, 'cloud': 0, 'feelslike_c': 14.7, 'feelslike_f': 58.4, 'windchill_c': 14.7, 'windchill_f': 58.4, 'heatindex_c': 14.6, 'heatindex_f': 58.4, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.1, 'gust_mph': 2.7, 'gust_kph': 4.3}}", 'score': 0.909123}, {'title': '

In [9]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4", model_provider="openai")

In [10]:
# calling a model

from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [11]:
model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [16]:
# expected call to tool
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Barcelona?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Barcelona'}, 'id': 'call_rooKv9DOOz6wN7q5KzUgmvE7', 'type': 'tool_call'}]


In [14]:
tools

[TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]